![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [32]:
import pandas as pd
import numpy as np

df = pd.read_csv("datasets/bank_marketing.csv")
print(df.columns)




Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')


In [33]:
client = df[['client_id','age','job','marital','education','credit_default','mortgage']].copy()
campaign = df[['client_id','number_contacts','contact_duration','previous_campaign_contacts','previous_outcome','campaign_outcome']].copy()
economics = df[['client_id','cons_price_idx','euribor_three_months']].copy()


In [34]:
client['job'] = client['job'].str.replace(".", "_", regex=False)
client['education'] = client['education'].str.replace(".", "_", regex=False)
client['education'] = client['education'].replace("unknown", np.nan)

In [35]:
client['credit_default'] = client['credit_default'].apply(lambda x: 1 if x == "yes" else 0)
client['credit_default'] = client['credit_default'].astype(bool)
print(client['credit_default'])

client['mortgage'] = client['mortgage'].apply(lambda x: 1 if x == "yes" else 0)
client['mortgage'] = client['mortgage'].astype(bool)

print(client['mortgage'])

0        False
1        False
2        False
3        False
4        False
         ...  
41183    False
41184    False
41185    False
41186    False
41187    False
Name: credit_default, Length: 41188, dtype: bool
0        False
1        False
2         True
3        False
4        False
         ...  
41183     True
41184    False
41185     True
41186    False
41187     True
Name: mortgage, Length: 41188, dtype: bool


In [36]:


campaign['previous_outcome'] = campaign['previous_outcome'].apply(lambda x: 1 if x == "success" else 0)
campaign['campaign_outcome'] = campaign['campaign_outcome'].apply(lambda x: 1 if x == "yes" else 0)


campaign['previous_outcome'] = campaign['previous_outcome'].astype(bool)
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype(bool)


print(campaign[['previous_outcome','campaign_outcome']])



       previous_outcome  campaign_outcome
0                 False             False
1                 False             False
2                 False             False
3                 False             False
4                 False             False
...                 ...               ...
41183             False              True
41184             False             False
41185             False             False
41186             False              True
41187             False             False

[41188 rows x 2 columns]


In [37]:
df["year"] = "2022"

df["day"] = df["day"].astype("str")

campaign["last_contact_date"] = df["year"] + "-" + df['month'] + "-" + df["day"]



campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], format = "%Y-%b-%d")

print (campaign["last_contact_date"])



0       2022-05-13
1       2022-05-19
2       2022-05-23
3       2022-05-27
4       2022-05-03
           ...    
41183   2022-11-30
41184   2022-11-06
41185   2022-11-24
41186   2022-11-17
41187   2022-11-23
Name: last_contact_date, Length: 41188, dtype: datetime64[ns]


In [38]:
client.dtypes

client_id          int64
age                int64
job               object
marital           object
education         object
credit_default      bool
mortgage            bool
dtype: object

In [39]:
campaign.dtypes

client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                        bool
campaign_outcome                        bool
last_contact_date             datetime64[ns]
dtype: object

In [40]:
economics.dtypes

client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object

In [41]:
client.to_csv('datasets/client.csv', index=False)
campaign.to_csv('datasets/campaign.csv', index=False)
economics.to_csv('datasets/economics.csv', index=False)